# DEPRECIATED

In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, BartTokenizer, BartForConditionalGeneration, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset, load_metric, load_from_disk
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import mlflow
import evaluate

In [2]:
batch_size = 32
max_input_length = 128
max_target_length = 128

cache_dir = "D:\\.cache"

In [4]:
# data = load_from_disk("./traintmp")
# df = data.to_pandas()
# df.drop(["text"], axis=1, inplace=True)
# df.dropna(inplace=True)



# train_df, val_df = train_test_split(df, test_size=0.3, random_state=0)
# val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=0)


In [3]:
def generate_dataset(dataframe, tokenizer):
    input_ids = []
    attention_masks = []
    labels = []

    for _, row in tqdm(dataframe.iterrows(), total=len(dataframe)):
        input_text = row["en"]
        target_text = row["jp"]
        
        model_inputs = tokenizer(
            text=input_text, max_length=max_input_length, padding=True, truncation=True
        )
        
        labels_temp = tokenizer(
            text_target=target_text, max_length=max_target_length, padding=True, truncation=True
        )
            
        input_ids.append(model_inputs["input_ids"])
        attention_masks.append(model_inputs["attention_mask"])
        labels.append(labels_temp["input_ids"])

    dataframe = dataframe.drop(columns=["en", "jp"])
    dataframe["input_ids"] = input_ids
    dataframe["attention_mask"] = attention_masks
    dataframe["labels"] = labels

    data = Dataset.from_pandas(dataframe, preserve_index=False)
    return data

In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base", cache_dir=cache_dir, use_fast=True, padding=True, truncation=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

In [17]:
metric = evaluate.load("bleu", cache_dir=cache_dir)

def process_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute(evaluation):
    preds, labels = evaluation

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    preds, labels = process_text(preds, labels)

    result = metric.compute(predictions=preds, references=labels)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    changed_result = {}
    for key, value in result.items():
        if type(value) == float:
            changed_result[key] = round(value, 4)
        else:
            changed_result[key] = round(np.mean(value), 4)
    
    return changed_result


In [18]:
# train_dataset = generate_dataset(train_df, tokenizer)
# val_dataset = generate_dataset(val_df, tokenizer)
# test_dataset = generate_dataset(test_df, tokenizer)

100%|██████████| 302522/302522 [05:56<00:00, 539.45it/s]


In [ ]:
# train_dataset.save_to_disk("./Mistral/train_dataset.pth")
# val_dataset.save_to_disk("./Mistral/val_dataset.pth")
# test_dataset.save_to_disk("./Mistral/test_dataset.pth")

In [6]:
train_dataset = load_from_disk("./Mistral/train_dataset.pth")
val_dataset = load_from_disk("./Mistral/val_dataset.pth")
test_dataset = load_from_disk("./Mistral/test_dataset.pth")

In [7]:
val_dataset = val_dataset.select(range(100))
test_dataset = test_dataset.select(range(100))

In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base", cache_dir=cache_dir)
model_args = Seq2SeqTrainingArguments(
    "./Mistral/args",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.02,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
    logging_dir="./Mistral/logs",
    save_steps=2000,
    save_strategy="steps",
    resume_from_checkpoint="./Mistral/args",
    generation_max_length=max_target_length,
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute
)

mlflow.set_experiment("BART")

c:\Users\Victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


<Experiment: artifact_location='file:///C:/Victo/NTNU/TDT4310---Project/mlruns/495351090728090601', creation_time=1712927809406, experiment_id='495351090728090601', last_update_time=1712927809406, lifecycle_stage='active', name='BART', tags={}>

In [19]:

trainer.train(resume_from_checkpoint=True)


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss,Bleu,Precisions,Brevity Penalty,Length Ratio,Translation Length,Reference Length,Gen Len
2,0.879300,0.822894,0.332000,0.363800,0.919200,0.922300,368.000000,399.000000,1.000000
3,0.828100,0.810006,0.340400,0.360200,0.948600,0.949900,379.000000,399.000000,1.000000
4,0.789600,0.755308,0.384000,0.452300,0.849400,0.859600,343.000000,399.000000,1.000000
5,0.748500,0.752316,0.362500,0.394500,0.921900,0.924800,369.000000,399.000000,1.000000
6,0.728200,0.715389,0.370100,0.400700,0.924600,0.927300,370.000000,399.000000,1.000000
7,0.706700,0.708748,0.331400,0.333000,1.000000,1.007500,402.000000,399.000000,1.000000
8,0.676400,0.692078,0.376400,0.379400,0.995000,0.995000,397.000000,399.000000,1.000000
9,0.664000,0.688385,0.330900,0.342000,0.972000,0.972400,388.000000,399.000000,1.000000
10,0.632100,0.674719,0.343500,0.380300,0.905600,0.909800,363.000000,399.000000,1.000000
11,0.613900,0.674300,0.371900,0.376300,0.992500,0.992500,396.000000,399.000000,1.000000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=661770, training_loss=0.572000113147044, metrics={'train_loss': 0.572000113147044, 'epoch': 15.0})

In [22]:
mlflow.end_run()

In [20]:
trainer.save_model("./Mistral/model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [21]:
test_res = trainer.predict(test_dataset)

In [30]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

pred = []
test_input = "私はチョコレートが大好きです。"
inputs = tokenizer(test_input, return_tensors="pt").to(device)
outputs = model.generate(**inputs)
pred.append(tokenizer.decode(outputs[0], skip_special_tokens=True))


c:\Users\Victo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [31]:
pred

['私はチョコレートが大好きで']